<a href="https://colab.research.google.com/github/Poojithavb/CMPE258-Project/blob/master/IDSGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings("ignore") 
import sys
import pandas as pd
from glob import glob
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

In [0]:
pwd = os.getcwd()
path = pwd + '/drive/My Drive/MachineLearningCVE'
dfs = []
filenames = glob(path + "/*.csv")
filenames.sort()
for filename in filenames:
    dfs.append(pd.read_csv(filename))
    
df = pd.concat(dfs, ignore_index=True)
df['Flow Bytes/s'] = df['Flow Bytes/s'].astype(float)
df[' Flow Packets/s'] = df[' Flow Packets/s'].astype(float)
df = df.fillna(0)
df = df.replace([np.inf, -np.inf], sys.maxsize)
#print("Columns which have NaN or NULLS: ")
#for col in df.columns:
#    print('%s: %d' % (col, df[col].isna().sum()))
substr = '�'
df[' Label'] = df[' Label'].str.replace(substr, '-')
encoding = {
        " Label": {"BENIGN": 0, "FTP-Patator": 1, "SSH-Patator": 1, 
                   "DoS slowloris": 1, "DoS Slowhttptest": 1, "DoS Hulk": 1, 
                   "DoS GoldenEye": 1, "Heartbleed": 1, 
                   "Web Attack - Brute Force" : 1, "Web Attack - XSS": 1,
                   "Web Attack - Sql Injection": 1, "Infiltration" : 1, 
                   "Bot":1, "DDoS":1,"PortScan": 1}
        }
df.replace(encoding, inplace=True)

In [0]:
full_data = df.drop(columns=[
    "Fwd Avg Bytes/Bulk",              
    " Fwd Avg Packets/Bulk",           
    " Fwd Avg Bulk Rate",              
    " Bwd Avg Bytes/Bulk",             
    " Bwd Avg Packets/Bulk", 
    "Bwd Avg Bulk Rate", 
    " Bwd PSH Flags", 
    " Bwd URG Flags"])

In [0]:
functional_features = ['Idle Mean',
 ' Idle Std',
 ' Idle Min',
 ' Idle Max',
 ' Avg Bwd Segment Size',
 ' Average Packet Size',
 ' URG Flag Count',
 'FIN Flag Count',
 ' Packet Length Variance',
 ' Packet Length Std',
 ' Packet Length Mean',
 ' Max Packet Length',
 ' Min Packet Length',
 ' Fwd IAT Max',
 ' Fwd IAT Std',
 ' Fwd IAT Mean',
 'Fwd IAT Total',
 ' Flow IAT Max',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Bwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Flow Duration']

In [0]:
functional_features = np.array([col in functional_features for col in full_data.columns[:-1]], dtype=np.float64)
nonFunctional_features = np.array([col not in functional_features for col in full_data.columns[:-1]], dtype=np.float64)

In [0]:
def _toarray(X, Y):
    X = X.astype(float)
    X = np.asarray(X)
    Y = np.asarray(Y)
    #Y = to_categorical(Y, 2)
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    return X, Y
    
def _getData(df):
    _all = df
    _norm = df.loc[df[' Label'] == 0]
    _mal = df.loc[df[' Label'] != 0]
    l_all = _all[' Label'][:,None]
    l_norm = _norm[' Label']
    l_mal = _mal[' Label']
    _all = _all.drop([' Label'], axis=1)
    _mal = _mal.drop([' Label'], axis=1)
    _norm = _norm.drop([' Label'], axis=1)
    data_all, label_all = _toarray(_all, l_all)
    data_mal, label_mal = _toarray(_mal, l_mal)
    data_norm, label_norm = _toarray(_norm, l_norm)
    return data_all, label_all, data_mal, label_mal, data_norm, label_norm

In [9]:
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(full_data.iloc[:,:-1], full_data.iloc[:,-1])
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 557646, 1: 557646})


In [0]:
dataframe = pd.DataFrame(X_res)
dataframe[' Label'] = y_res 

In [0]:
X_all, Y_all, malicious_data, Y_mal, normal_data, Y_norm = _getData(dataframe)

In [12]:
print("Malicious_data",malicious_data.shape)
print("Normal_data",normal_data.shape)

Malicious_data (557646, 70)
Normal_data (557646, 70)


In [0]:
X_train, X_test, y_train,y_test = train_test_split(X_all,Y_all,test_size=0.1)

In [14]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [15]:
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
y_pred = clf.predict(X_test)
print("F1 Score: ", f1_score(y_test, y_pred, average='micro'))
print("MSE: ", mean_squared_error(y_test, y_pred))

F1 Score:  0.9986819689769568
MSE:  0.0013180310230431274


In [0]:
def _generator(input=(70,)):
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, input_shape=(70,)))
    model.add(layers.BatchNormalization(momentum=0.8))
    #model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization(momentum=0.8))
    #model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization(momentum=0.8))
    #model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(np.prod(70,)))
    #model.add(layers.Activation('sigmoid'))
    
    model.summary()
    
    return model

In [33]:
g = _generator()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                2272      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32)                128       
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_8 (Batch (None, 64)                256       
_________________________________________________________________
dense_11 (Dense)             (None, 128)               8320      
_________________________________________________________________
batch_normalization_9 (Batch (None, 128)               512       
_________________________________________________________________
dense_12 (Dense)             (None, 256)              

In [0]:
def _discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_shape=(70, )))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    #model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Dense(1))
    
    model.summary()
    
    return model

In [35]:
d = _discriminator()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               9088      
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)               512       
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
batch_normalization_12 (Batc (None, 128)               512       
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)               

In [0]:
def get_batch(data, batch_size):
    indices = np.random.randint(low=0, high=len(data), size=batch_size)
    np.random.shuffle(indices)
    batch_data = data[indices]
    return batch_data

In [0]:
def generate_latent_points(latent_dim, n):
    x_input = np.random.randn(latent_dim * n)
    x_input = x_input.reshape(n, latent_dim)
    return x_input

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(outputs):
    return cross_entropy(tf.zeros_like(outputs), outputs)

In [0]:
def discriminator_loss(ids_predicts, d_predicts):    
    return cross_entropy(ids_predicts, d_predicts)

In [0]:
def ids_compute(ids_classifer, ids_samples, batch_size):
    a = tf.make_tensor_proto(ids_samples)
    ids_samples = tf.make_ndarray(a)
    ids_samples[ids_samples > 0.5] = 1
    ids_samples[ids_samples < 0.5] = 0
    ids_preds = ids_classifer.predict(ids_samples)
    mse = mean_squared_error(np.hstack([np.ones(int(len(ids_preds)/2)), np.zeros(int(len(ids_preds)/2))]), ids_preds)
    ids_predictions = tf.convert_to_tensor(ids_preds, float)
    ids_samples = tf.convert_to_tensor(ids_samples, float)

    return ids_samples, ids_predictions, mse

In [0]:
def train(epochs=10, batch_size=10000, functional_features=functional_features,
          nonFunctional_features=nonFunctional_features):
    
    losses = []
    ids, generator, discriminator = clf, g, d
    generator_in = generate_latent_points(70, batch_size)
    
    generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
    discriminator_optimizer = tf.keras.optimizers.RMSprop(1e-2)
    
    for e in range(epochs):
        for i in range(int(normal_data.shape[0]/batch_size)):
            normal_samples = get_batch(normal_data, batch_size=batch_size)
            malicious_samples = get_batch(malicious_data, batch_size=batch_size)
                        
            normal_samples = tf.convert_to_tensor(normal_samples, float)
            malicious_samples = tf.convert_to_tensor(malicious_samples, float)
            functional_features = tf.convert_to_tensor(functional_features, float)
            nonFunctional_features = tf.convert_to_tensor(nonFunctional_features, float)
            generator_in = tf.convert_to_tensor(generator_in, float)


            
            with tf.GradientTape() as gTape, tf.GradientTape() as dTape:
                generated_maliciousSamples = tf.math.add(tf.math.multiply(malicious_samples, 
                                                                          functional_features),
                                                    tf.math.multiply(generator(generator_in), 
                                                                     nonFunctional_features))
            
                ids_samples = tf.concat([generated_maliciousSamples, normal_samples], 0)
#                 ids_samples = tf.Session().run(ids_samples)
#                 ids_predictions = ids.predict(ids_samples)
                ids_samples, ids_predictions, ids_loss = ids_compute(ids, ids_samples, batch_size=batch_size)
                #print("ids loss:", ids_loss)
                     
                d_predictions = discriminator(ids_samples)
            
                d_loss = discriminator_loss(ids_predictions, d_predictions)*100000
                #print("dloss:", d_loss)


                disGenOut = discriminator(generated_maliciousSamples)
            
                g_loss = generator_loss(disGenOut)*1000000
                #print("gloss:", g_loss)
    
            gGradients = gTape.gradient(g_loss, generator.trainable_variables)
            dGradients = dTape.gradient(d_loss, discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(gGradients, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(dGradients, discriminator.trainable_variables))
            losses.append((d_loss, g_loss))

        print("Epoch:{:>3}/{} Discriminator Loss:{:>7.4f} Generator Loss:{:>7.4f} IDS Loss:{:>7.4f}".format(
            e+1, epochs, d_loss, g_loss, ids_loss))            

In [44]:
train()

Epoch:  1/10 Discriminator Loss:17.3396 Generator Loss:39.2695 IDS Loss: 0.5038
Epoch:  2/10 Discriminator Loss: 2.7797 Generator Loss: 0.1766 IDS Loss: 0.5041
Epoch:  3/10 Discriminator Loss: 0.0178 Generator Loss: 0.0016 IDS Loss: 0.5047
Epoch:  4/10 Discriminator Loss: 0.2519 Generator Loss: 0.0869 IDS Loss: 0.5050
Epoch:  5/10 Discriminator Loss: 2.7789 Generator Loss: 0.0000 IDS Loss: 0.5040
Epoch:  6/10 Discriminator Loss:20.4633 Generator Loss: 0.0000 IDS Loss: 0.5046
Epoch:  7/10 Discriminator Loss: 9.4784 Generator Loss: 0.0000 IDS Loss: 0.5054
Epoch:  8/10 Discriminator Loss: 0.2956 Generator Loss: 0.0000 IDS Loss: 0.5050
Epoch:  9/10 Discriminator Loss: 0.0141 Generator Loss: 0.0000 IDS Loss: 0.5044
Epoch: 10/10 Discriminator Loss: 5.9453 Generator Loss: 0.0000 IDS Loss: 0.5038
